In [11]:
import torch.nn.functional as F
import voc12.dataloader
import numpy as np
import argparse
import torch
import os
from PIL import Image
from torch.utils.data import DataLoader
from misc import pyutils, torchutils
from net.resnet50_cam import CAM, Net, ReCAM    
from misc import imutils
import matplotlib.pyplot as plt
from tqdm import tqdm

In [7]:
train_list = '/Users/Andy/Projects/c2am/voc12/train_aug.txt'
voc12_root = '/Users/Andy/Projects/dataset/VOCdevkit/VOC2012'
baseline_cam_weight_path = '/Users/Andy/Projects/resnet50_baseline_512.pth'

cls_dataset = voc12.dataloader.VOC12ClassificationDataset(train_list, voc12_root=voc12_root,
                                                            resize_long=(320, 640), hor_flip=True,
                                                            crop_size=512, crop_method="random")
cls_data_loader = DataLoader(cls_dataset,
                               batch_size=1,
                               shuffle=False,
                               num_workers=0)


cam_dataset = voc12.dataloader.VOC12ClassificationDatasetMSF(train_list,
                                                          voc12_root=voc12_root,
                                                          scales=(1.0, 0.5, 1.5, 2.0))
cam_data_loader = DataLoader(cam_dataset, batch_size=1, shuffle=False)

cls_model = Net()
cam_model = CAM()
cls_model.eval()
cam_model.eval()
cls_model.load_state_dict(torch.load(baseline_cam_weight_path, map_location='cpu'))
cam_model.load_state_dict(torch.load(baseline_cam_weight_path, map_location='cpu'))

<All keys matched successfully>

In [25]:
for pack in tqdm(cls_data_loader):
    names = pack['name']
    imgs = pack['img']
    labels = pack['label']
    if labels[0][11] and labels[0][14] == 0:
        logits, _ = cls_model(imgs)
        probs = F.softmax(logits, dim=1)
        if probs[0][14] > 0.5:
            print(probs)
            print(names)

 68%|██████▊   | 7145/10582 [15:22<09:17,  6.16it/s]  

tensor([[3.2235e-08, 2.4940e-06, 1.1918e-07, 3.8905e-07, 1.6577e-04, 2.4210e-07,
         4.1109e-06, 1.4293e-04, 4.3902e-05, 1.9703e-07, 2.2379e-06, 2.9050e-01,
         1.1265e-06, 8.3566e-08, 6.9488e-01, 2.4513e-05, 1.4147e-07, 2.5782e-05,
         2.2290e-07, 1.4211e-02]], grad_fn=<SoftmaxBackward0>)
['2010_001099']


100%|██████████| 10582/10582 [26:47<00:00,  6.58it/s]


In [3]:
label_set = {}

for pack in data_loader:
    label = tuple(*pack['label'].numpy().tolist())
    if label not in label_set:
        label_set[label] = 0
    label_set[label] += 1

KeyboardInterrupt: 

In [ ]:
for k,v in label_set.items():
    if k[5] or k[14]:
        print(k, v)

In [ ]:
horse_wrong = []

for pack in data_loader:
    

In [33]:
import numpy as np

CAT_LIST = ['aeroplane', 'bicycle', 'bird', 'boat',
            'bottle', 'bus', 'car', 'cat', 'chair',
            'cow', 'diningtable', 'dog', 'horse',
            'motorbike', 'person', 'pottedplant',
            'sheep', 'sofa', 'train',
            'tvmonitor']
idx = np.asarray((0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0))
print(np.nonzero(idx))
print(CAT_LIST[14])

(array([ 5, 14]),)
person


In [ ]:
from PIL import Image
for pack in data_loader:
    label = pack['label'].numpy().tolist()[0]
    size = pack['size']
    if label[5] or label[14]:
        with torch.no_grad():
            strided_up_size = imutils.get_strided_up_size(size, 16)
            baseline_outputs = [model_baseline(img[0]) for img in pack['img']]
            baseline_highres_cam = [F.interpolate(torch.unsqueeze(
                        o, 1), strided_up_size, mode='bilinear', align_corners=False) for o in baseline_outputs]
            baseline_highres_cam = torch.sum(torch.stack(baseline_highres_cam, 0), 0)[
                        :, 0, :size[0], :size[1]]
            valid_cat = torch.nonzero(pack['label'][0])[:, 0]
            baseline_highres_cam /= F.adaptive_max_pool2d(baseline_highres_cam, (1, 1)) + 1e-5

        w = size[0].item()
        h = size[1].item()
        fig = plt.figure(figsize=(192.0,108.0))
        columns = 20 + 1
        rows = 1
        n = 20
        for i in range(1, n +1):
            img = baseline_highres_cam[i-1]
            img = img.unsqueeze(0).unsqueeze(0)
            fig.add_subplot(rows, columns, i)
            plt.imshow(img.numpy().squeeze())
            plt.title(CAT_LIST[i-1])
        fig.add_subplot(rows, columns, i+1)
        plt.imshow(Image.open(os.path.join(voc12_root, 'JPEGImages') + '/' + pack['name'][0] + '.jpg'))
        plt.title(pack['name'][0])
        fig.savefig('../cam_vis/{}.png'.format(pack['name'][0]), dpi=fig.dpi)

/Users/Andy/Projects/front_door_cam/.env/lib/python3.6/site-packages/ipykernel_launcher.py:18: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
